In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [20]:
# Get geometry data from Natural Earth
url = "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip"
data_v1 = gpd.read_file(url)
print(data_v1.head())

        featurecla  scalerank  LABELRANK SOVEREIGNT SOV_A3  ADM0_DIF  LEVEL  \
0  Admin-0 country          0          2  Indonesia    IDN         0      2   
1  Admin-0 country          0          3   Malaysia    MYS         0      2   
2  Admin-0 country          0          2      Chile    CHL         0      2   
3  Admin-0 country          0          3    Bolivia    BOL         0      2   
4  Admin-0 country          0          2       Peru    PER         0      2   

                TYPE TLC      ADMIN  ... FCLASS_TR  FCLASS_ID FCLASS_PL  \
0  Sovereign country   1  Indonesia  ...       NaN        NaN       NaN   
1  Sovereign country   1   Malaysia  ...       NaN        NaN       NaN   
2  Sovereign country   1      Chile  ...       NaN        NaN       NaN   
3  Sovereign country   1    Bolivia  ...       NaN        NaN       NaN   
4  Sovereign country   1       Peru  ...       NaN        NaN       NaN   

  FCLASS_GR  FCLASS_IT FCLASS_NL FCLASS_SE  FCLASS_BD FCLASS_UA  \
0      

In [21]:
# Set SRS=4326
data_v1 = data_v1.to_crs(epsg=4326)
print(data_v1.crs)

EPSG:4326


In [22]:
# Read the ISO_A2 list file
countries = pd.read_csv('Task-1-Data/iso_a2_list 2.csv')
# Get rid of all the nan rows
countries = countries.dropna(how="all")
# Split the first colmn
countries['iso_a2'] = countries['iso_a2;country_name_eng'].str.split(';').apply(lambda x: x[0])
countries['name_part2'] = countries['iso_a2;country_name_eng'].str.split(';').apply(lambda x: x[1])
# Get the full name of the country
countries['name'] = countries['Unnamed: 1'].fillna('') + ' ' + countries['name_part2']
#captionise the iso_a2 code
countries['iso_a2'] = countries['iso_a2'].str.upper()
print(countries)

     iso_a2;country_name_eng Unnamed: 1 iso_a2            name_part2  \
1    ae;United Arab Emirates        NaN     AE  United Arab Emirates   
3             af;Afghanistan        NaN     AF           Afghanistan   
5     ag;Antigua and Barbuda        NaN     AG   Antigua and Barbuda   
7                ai;Anguilla        NaN     AI              Anguilla   
9                 al;Albania        NaN     AL               Albania   
..                       ...        ...    ...                   ...   
491              vn;Viet Nam        NaN     VN              Viet Nam   
493                 ws;Samoa        NaN     WS                 Samoa   
495               vu;Vanuatu        NaN     VU               Vanuatu   
497                 ye;Yemen        NaN     YE                 Yemen   
499               yt;Mayotte        NaN     YT               Mayotte   

                      name  
1     United Arab Emirates  
3              Afghanistan  
5      Antigua and Barbuda  
7                 A

In [23]:
# Delete the columns without proprt ISO_A2 values
#data_v3 = data_v2[data_v2['ISO_A2_EH'] != '-99']
#print(data_v3)
# Match the geometry with the ISO_A2 list
data_v2 = data_v1.merge(countries, left_on='ISO_A2_EH',right_on='iso_a2')
# Keep only full name, ISO_A2, geometry, populatio and GDP of the table
data_v3 = data_v2[['geometry', 'name', 'iso_a2','POP_EST','GDP_MD']]
print(data_v3)

                                              geometry  \
0    MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...   
1    MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...   
2    MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...   
3    POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...   
4    MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...   
..                                                 ...   
240  MULTIPOLYGON (((134.27149 7.07453, 134.27931 7...   
241  POLYGON ((144.88640 13.64020, 144.89666 13.617...   
242  MULTIPOLYGON (((145.20574 14.18138, 145.25245 ...   
243  MULTIPOLYGON (((50.55161 26.19424, 50.59474 26...   
244  MULTIPOLYGON (((113.55860 22.16303, 113.56943 ...   

                                name iso_a2      POP_EST   GDP_MD  
0                          Indonesia     ID  270625568.0  1119190  
1                           Malaysia     MY   31949777.0   364681  
2                              Chile     CL   18952038.0   282318  
3     Plurinational State of Bo

In [24]:
# check validity of data
validity = data_v3.is_valid

if validity.all():
    print('All geometries are valid')
# if some geometry is not valid, correct the geometry by setting the buffer
else:
    print('At least one geometry is not valid')
    # get the index where validity is False
    invalid_row_idx = np.where(validity == False)[0]
    print('Invalid row index:', invalid_row_idx)
    # locate the invalid geometries and set the buffer
    selected_rows = data_v3.iloc[invalid_row_idx] 
    selected_rows['geometry'] = selected_rows['geometry'].buffer(0)
    data_v3.iloc[invalid_row_idx] = selected_rows
    # reexmaine the validity
    validity_v2=data_v3.is_valid
    if validity_v2.all():
        print('the invalid geometry has been corrected, and all geometries are valid')
    else:
        print('Still at least one geometry is not valid')

At least one geometry is not valid
Invalid row index: [160]


c:\Users\risen\.conda\envs\Python 38\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


the invalid geometry has been corrected, and all geometries are valid


In [25]:
# Export the data to geopackage
data_v3.to_file('Task-2-Data/Task-1-Exported-Geomeries.gpkg', layer='ISO_A2_Geometries', driver="GPKG")